In [1]:
#Ingestion

def leggi_file_txt(nome_file):
    out = []
    with open(nome_file, 'r') as file:
        for riga in file:
                out.append(list(riga[:-1]))
    return out

nome_file = 'input_day20.txt'
nome_file_test= 'input_day20_test.txt'
data= leggi_file_txt(nome_file)
data_test=leggi_file_txt(nome_file_test)

In [2]:
# Funzioni parte 1

import networkx as nx
from functools import cache



def crea_grafo(lista):
    G = nx.Graph()
    punti_buoni=['.','S','E']
    for i in range(len(lista)):
        for j in range(len(lista[0])):
            if lista[i][j] in punti_buoni:
                G.add_node((i,j))
    for i in range(1,len(lista)-1):
        for j in range(1,len(lista[0])-1):
            if lista[i][j] in punti_buoni and lista[i][j+1] in punti_buoni :
                G.add_edge((i,j), (i,j+1))
            if lista[i][j] in punti_buoni  and lista[i][j-1] in punti_buoni :
                G.add_edge((i,j), (i,j-1))
            if lista[i][j] in punti_buoni  and lista[i+1][j] in punti_buoni :
                G.add_edge((i,j), (i+1,j))
            if lista[i][j] in punti_buoni  and lista[i-1][j] in punti_buoni :
                G.add_edge((i,j), (i-1,j))
    return G

def trova_inizio(lista):
    for i in range(len(lista)):
        for j in range(len(lista[0])):
            if lista[i][j]=='S':
                return (i,j)
    return 0

def trova_fine(lista):
    for i in range(len(lista)):
        for j in range(len(lista[0])):
            if lista[i][j]=='E':
                return (i,j)
    return 0

def percorso_minimo(G, S, E):
    out_1=nx.dijkstra_path(G, S, E)
    out_2=nx.dijkstra_path_length(G,S,E)
    return out_1, out_2



def aggiungi_nodo(G, nodo, lista):
    punti_buoni=['.','S','E']
    G.add_node(nodo)
    if lista[nodo[0]+1][nodo[1]] in punti_buoni:
            G.add_edge(nodo, (nodo[0]+1, nodo[1]))
    if lista[nodo[0]-1][nodo[1]] in punti_buoni:
            G.add_edge(nodo, (nodo[0]-1, nodo[1]))
    if lista[nodo[0]][nodo[1]+1] in punti_buoni:
            G.add_edge(nodo, (nodo[0], nodo[1]+1))
    if lista[nodo[0]][nodo[1]-1] in punti_buoni:
            G.add_edge(nodo, (nodo[0], nodo[1]-1))
    return G


def rimuovi_nodo(G, nodo, lista):
    if lista[nodo[0]][nodo[1]]=='#':
        G.remove_node(nodo)
    return G



def lista_cheat(percorso, lista):
    out=[]
    for coordinata in percorso:
        if coordinata[0]+1<len(lista)-1 and lista[coordinata[0]+1][coordinata[1]]=='#':
            out.append((coordinata[0]+1,coordinata[1]))
        if coordinata[0]-1>0  and lista[coordinata[0]-1][coordinata[1]]=='#' :
            out.append((coordinata[0]-1,coordinata[1]))
        if coordinata[1]+1<len(lista[0])-1 and lista[coordinata[0]][coordinata[1]+1]=='#':
            out.append((coordinata[0],coordinata[1]+1))
        if coordinata[1]-1>0 and lista[coordinata[0]][coordinata[1]-1]=='#' :
            out.append((coordinata[0],coordinata[1]-1))
    return out

def aggiungi_nodi(G, nodo_1, nodo_2, lista):
    punti_buoni=['.','S','E']
    G.add_node(nodo_1)
    G.add_node(nodo_2)
    G.add_edge(nodo_1,nodo_2)
    nodi=[nodo_1, nodo_2]
    for nodo in nodi:
        if lista[nodo[0]+1][nodo[1]] in punti_buoni:
            G.add_edge(nodo, (nodo[0]+1, nodo[1]))
        if lista[nodo[0]-1][nodo[1]] in punti_buoni:
            G.add_edge(nodo, (nodo[0]-1, nodo[1]))
        if lista[nodo[0]][nodo[1]+1] in punti_buoni:
            G.add_edge(nodo, (nodo[0], nodo[1]+1))
        if lista[nodo[0]][nodo[1]-1] in punti_buoni:
            G.add_edge(nodo, (nodo[0], nodo[1]-1))
    return G


def rimuovi_nodi(G, nodo_1, nodo_2, lista):
    if lista[nodo_1[0]][nodo_1[1]]=='#':
        G.remove_node(nodo_1)
    if lista[nodo_2[0]][nodo_2[1]]=='#':
        G.remove_node(nodo_2)
    return G

def lista_cheat_2(mappa, percorso):
    out=[]
    for coordinata in percorso:
        if coordinata[0]+1<len(mappa)-1 and coordinata[0]+2<len(mappa)-1:
            out.append([(coordinata[0]+1,coordinata[1]),(coordinata[0]+2,coordinata[1])])
        if coordinata[0]-1>0  and coordinata[0]-2>0:
            out.append([(coordinata[0]-1,coordinata[1]),(coordinata[0]-2,coordinata[1])])
        if coordinata[1]+1<len(mappa[0])-1 and coordinata[1]+2<len(mappa[0])-1:
            out.append([(coordinata[0],coordinata[1]+1),(coordinata[0],coordinata[1]+2)])
        if coordinata[1]-1>0  and coordinata[1]-2>0:
            out.append([(coordinata[0],coordinata[1]-1),(coordinata[0],coordinata[1]-2)])
    return out

def rimuovi_duplicati(lista):
    out=[]
    for item in lista:
        if item not in out:
            out.append(item)
    return out


In [4]:
#Calcolo parte 1
from tqdm import tqdm
answer_1=0
mappa=data

grafo=crea_grafo(mappa)
inizio=trova_inizio(mappa)
fine=trova_fine(mappa)
percorso, lunghezza_standard=percorso_minimo(grafo, inizio, fine)

cheats=rimuovi_duplicati(lista_cheat(percorso, mappa))


vantaggi=[]
for i in tqdm(range(len(cheats))):
    aggiungi_nodo(grafo,cheats[i], mappa)
    app_1, app_2=percorso_minimo(grafo, inizio, fine)
    if lunghezza_standard-app_2>0:
        vantaggi.append(lunghezza_standard-app_2)
    rimuovi_nodo(grafo,cheats[i], mappa)


for item in vantaggi:
    if item>=100:
        answer_1+=1

print(answer_1)

100%|██████████| 9542/9542 [1:45:19<00:00,  1.51it/s]

1382
